In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import time
import gspread
import pandas as pd
import os
import concurrent.futures
from google.oauth2 import service_account
from googleapiclient import errors
from googleapiclient.discovery import build
import google.auth
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from selenium.common.exceptions import TimeoutException
import clipboard
import re
from googleapiclient.discovery import build
from google.auth import exceptions


### Google Sheet

- Authentication
- read worksheet
- write worksheet
- Get the worksheet data in dataframe


In [ ]:
# authenticate and access google sheet
gc = gspread.service_account(filename="./credentials.json")
sh = gc.open_by_url(
    "Sheet link"
)

In [ ]:
# funtion to read data from google sheet
def readWorkSheet(name):
    worksheet = sh.worksheet(name)
    data = worksheet.get_all_values()
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    return df

In [ ]:
# function to write data in google sheet
def writeInWorkSheet(workShetName, link, index, col_index):
    worksheet = sh.worksheet(workShetName)
    worksheet.update_cell(index, col_index, link)

In [ ]:
dfTitleSheet = readWorkSheet("Article Generation")

In [ ]:
dfTitleSheet

In [ ]:
dfOutlineSheet = readWorkSheet("Frase Generation")

In [ ]:
dfOutlineSheet

### Google Docs


In [ ]:
def create_google_doc_with_response(title, response):
    # If modifying these scopes, delete the file token.json.
    SCOPES = [
        "https://www.googleapis.com/auth/documents",
        "https://www.googleapis.com/auth/drive",
    ]

    # Authenticate using the credentials.json file
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "client_secret.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("docs", "v1", credentials=creds)
        drive_service = build("drive", "v3", credentials=creds)

        # Create a new Google Doc with the specified title
        body = {"title": f"{title} - Generated Response"}
        doc = service.documents().create(body=body).execute()

        # Update sharing permissions to allow anyone with the link to view the document
        file_id = doc.get("documentId")
        permissions = {"role": "writer", "type": "anyone"}

        try:
            drive_service.permissions().create(
                fileId=file_id, body=permissions
            ).execute()
            print(f"Sharing permissions updated for the document: {doc.get('title')}")
        except errors.HttpError as error:
            print(f"An error occurred while updating sharing permissions: {error}")

        # Get the document link
        document_id = doc.get("documentId")
        document_link = f"https://docs.google.com/document/d/{document_id}/edit"

        print(f"Created document with title: {doc.get('title')}")
        print(f"Document link: {document_link}")

        print("The title of the document is: {}".format(doc.get("title")))
        # Write the response to the Google Doc
        requests = [
            {
                "insertText": {
                    "location": {
                        "index": 1,
                    },
                    "text": response,
                }
            }
        ]

        try:
            result = (
                service.documents()
                .batchUpdate(documentId=doc["documentId"], body={"requests": requests})
                .execute()
            )
            print(f"Response added to the Google Doc: {doc.get('title')}")
        except errors.HttpError as error:
            print(f"An error occurred: {error}")
            result = None

        print(result)
        return document_link
    except HttpError as err:
        print(err)

In [ ]:
def get_google_doc_id_from_url(url):
    """
    This function takes a Google Document URL as an argument,
    extracts the Document ID from the URL, and returns it.

    :param url: The URL of the Google Document.
    :return: A string containing the Document ID.
    """
    match = re.search(r'd/([a-zA-Z0-9-_]+)/', url)
    if match:
        return match.group(1)
    else:
        print(f"Could not extract document ID from URL: {url}")
        return None


In [ ]:
def get_all_text_from_google_doc(url):
    """
    This function takes a Google Document ID and credentials as arguments,
    extracts all text from the document, and returns it as a single string.

    :param document_id: The ID of the Google Document.
    :param creds: The credentials needed to access the Google Document.
    :return: A string containing all text from the Google Document.
    """
    SCOPES = [
        "https://www.googleapis.com/auth/documents",
        "https://www.googleapis.com/auth/drive",
    ]

    # Authenticate using the credentials.json file
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "client_secret.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    
    # Get Document ID from URL
    document_id = get_google_doc_id_from_url(url)
    try:
        # Use the Google Docs API.
        docs_service = build("docs", "v1", credentials=creds)
        doc = docs_service.documents().get(documentId=document_id).execute()

        # Extract all text from the document.
        text_content = ""
        for element in doc["body"]["content"]:
            if "paragraph" in element:
                for text_element in element["paragraph"]["elements"]:
                    text_run = text_element.get("textRun")
                    if text_run:
                        text_content += text_run.get("content")

        return text_content

    except exceptions.GoogleAuthError as e:
        print(f"Failed to access the document: {e}")
        return None

# Main Code

### Check For Verified Outlines

In [ ]:
def check_for_verified_outlines():
    for index, row in dfTitleSheet.iterrows():
        if(dfTitleSheet['Live Link'][index] != ''):
            print('Already Generated')
            continue
        if(dfTitleSheet['Outline'][index] != ''):
            print('Already Copied')
            continue
        title = row["Titles"]
        rows = dfOutlineSheet.loc[dfOutlineSheet["Titles"] == title]
        print(index)
        print(rows["Verified"][index])
        if rows["Verified"][index] == "Yes":
            print('Copying outline to Google Doc...')
            # Get the document link
            link = rows['Live Link'][index]
            # Get the outline from document
            print('Getting outline from Google Doc...')
            text = get_all_text_from_google_doc(link)
            
            # Get Outline Column
            col_index = dfTitleSheet.columns.tolist().index("Outline")
            writeInWorkSheet("Article Generation", text, index + 2, col_index + 1)
            print('Outline copied to Google Sheet for' + title)
        else:
            print('Outline not verified for ' + title + '. Skipping...')
            continue


In [ ]:
check_for_verified_outlines()

In [ ]:
dfTitleSheet = readWorkSheet("Article Generation")

In [ ]:
dfTitleSheet

## Login
-  Should only be called once

In [ ]:
driver = uc.Chrome()

In [ ]:
def login(username, password):
    # 1. Open the browser and visit the website
    driver.get("https://chat.openai.com/?model=gpt-4")

    # 2. Press the login button
    element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located(
            (
                By.XPATH,
                '//div[contains(text(), "Log in")]/ancestor::button[contains(@class, "relative flex h-12 items-center justify-center rounded-md text-center text-base font-medium bg-black/5 hover:bg-black/10 dark:bg-[rgba(255,255,255,0.15)] dark:hover:bg-white/20")]',
            )
        )
    )
    driver.execute_script("arguments[0].click();", element)
    # 3. Enter email address and press Continue button
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "username"))
    ).send_keys(username)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//button[@type="submit"]'))
    ).click()

    # 4. Enter password and press Continue button
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "password"))
    ).send_keys(password)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div/main/section/div/div/div/form/div[3]/button'))
    ).click()
    # # 5. Close all popups if they exist
    while True:
        try:
            # WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div/div/div/div[2]/div/div[2]/button'))).click()
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located(
                    (
                        By.XPATH,
                        '//button[contains(@class, "btn relative btn-neutral ml-auto")]//div[contains(text(), "Next")]',
                    )
                )
            ).click()
        except:
            break
    WebDriverWait(driver, 3).until(
        EC.presence_of_element_located(
            (
                By.XPATH,
                '//button[contains(@class, "btn relative btn-primary ml-auto")]//div[contains(text(), "Done")]',
            )
        )
    ).click()

In [ ]:
login("username", 'password')

In [ ]:
# Iterate over the rows in the DataFrame
for index, row in dfTitleSheet.iterrows():
    if dfTitleSheet["Live Link"][index] != "":
        print("Already Generated")
        continue
    # Get Prompt from the DataFrame
    prompt = row["Prompt"]
    # Replace newline characters with space
    prompt = prompt.replace("\n", " ")

    # Click on the 'New Chat' button
    WebDriverWait(driver, 3).until(
        EC.presence_of_element_located(
            (
                By.XPATH,
                "/html/body/div[1]/div[1]/div[1]/div/div/div/nav/div[1]/a",
            )
        )
    ).click()

    # Click on selecting the 'GPT-4' model
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located(
            (
                By.XPATH,
                "/html/body/div[1]/div[1]/div[2]/div/main/div[1]/div/div/div/div[1]/div/div/ul/li[2]/button",
            )
        )
    ).click()

    # 7. Focus on text-area and enter some random text
    text_area = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "prompt-textarea"))
    )
    print(prompt)
    text_area.click()
    text_area.send_keys(prompt)
    text_area.send_keys(Keys.RETURN)

    # Check if 'Continue generating' button appears and click it
    # If 'Continue generating' doesn't appear, but 'Regenerate response' does, click 'Copy' button
    print("Waiting for generating to finish...")
    while True:
        try:
            # Look for 'Stop generating' button
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located(
                    (
                        By.XPATH,
                        '//button[contains(@class, "btn relative btn-neutral whitespace-nowrap border-0 md:border")]//div[contains(text(), "Stop generating")]',
                    )
                )
            )
            # print("Continuing...")
            # continue
        except:
            try:
                # Look for 'Regenerate response' button, if found then click 'Copy' button
                regenerate_button = WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located(
                        (
                            By.XPATH,
                            '//button[contains(@class, "btn relative btn-neutral whitespace-nowrap -z-0 border-0 md:border")]//div[contains(text(), "Regenerate")]',
                        )
                    )
                )
                # Copy Button
                print("copying...")
                # copy_button = WebDriverWait(driver, 3).until(
                #     EC.element_to_be_clickable(
                #         (
                #             By.XPATH,
                #             '//button[contains(@class, "flex ml-auto gap-2 rounded-md p-1 hover:bg-gray-100 hover:text-gray-700 dark:text-gray-400 dark:hover:bg-gray-700 dark:hover:text-gray-200 disabled:dark:hover:text-gray-400")]',
                #         )
                #     )
                # ).click()
                time.sleep(10)
                element = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[2]/div/main/div[1]/div/div/div/div[2]/div/div[2]/div[2]/div/button')
                driver.execute_script("arguments[0].click();", element)

                print("Copied response to clipboard.")
                break
            except TimeoutException:
                print(
                    "Neither 'Continue generating', 'Stop generating', nor 'Regenerate response' buttons were found."
                )
                exit(1)

    # Get the text copied in the clipboard
    copied_text = clipboard.paste()
    print("Text Copied from clipboard")
    doc_link = ""
    try:
        # Use the copied text as the response in your Google Docs function
        doc_link = create_google_doc_with_response(row["Titles"], copied_text)
        print("Created Google Doc with response.")
        print("doc_link: ", doc_link)
    except Exception as e:
        print("Error creating Google Doc with response.")
        print(e)

    # Get Live Link Column
    col_index = dfTitleSheet.columns.tolist().index("Live Link")
    writeInWorkSheet("Article Generation", doc_link, index + 2, col_index + 1)

    # Get Live Link Column
    col_index = dfTitleSheet.columns.tolist().index("Status")
    writeInWorkSheet("Article Generation", "Generated", index + 2, col_index + 1)